## Functionality that needs to be added

- ~~Trim Raw Input~~
- ~~Night vs Day bouts (critera - choose where over 50% of bout is spent, in case of tie choose night)~~
- Create Sleep Profile and summary stats
- Create function to run all flies and arrange in exportable format
- Write to seperate sheets in Excel

### Import Libraries that will be needed

In [1]:
import pandas as pd
import datetime

### Import data from .csv

In [2]:
df_raw = pd.read_csv("Bethany_Raw.csv")

### Trim to 24 hour range

In [3]:
lower_limit = datetime.time(10,0,0)
upper_limit = datetime.time(10,5,0)
def get_data_range(L, lower_limit, upper_limit):
    """
    Find the start and end indices for a 24 hour range starting between a given time range
    Takes arguments L - a list of datetime.time objects, lower_limit and upper_limit - datetime.time objects 
    to specificy the range in which to choose the start time
    Returns a tuple: (start_index, end_index)
    """
    start_index = 0
    for i in range(len(L)):
        if L[i] >= lower_limit and L[i] < upper_limit:
            start_index = i
            break
    return (start_index, start_index + 288)


In [4]:
# Test trim on raw data
df_raw.Time = pd.to_datetime(df_raw.Time, format='%H:%M:%S').dt.time
trim_index = get_data_range(list(df_raw.Time), lower_limit, upper_limit)
df_trim = df_raw[trim_index[0]:trim_index[1]]
df_trim = df_trim.reset_index(drop=True)

### Make a column of bools to indicate day vs night

In [5]:
def isNight(x):
    night_start = datetime.time(22,0,0)
    night_end = datetime.time(10,0,0)
    if (x >= night_start and x < datetime.time(23, 59, 59)) or x < night_end:
        return True
    else:
        return False

In [6]:
df_trim.loc[:,'isNight'] = df_trim.Time.apply(isNight)

### Generate bout indices

In [7]:
# Script for collecting bouts
# Will need to create a list of bout indices to access resp data during bouts
def get_bout_indices(L):
    """
    Takes a list L and returns a list of tuples of the start/end indices in which sleeping bouts occurred.
    I.e. if two sleeping bouts occured, the first from index 5 to 20, and the second from index 30 to 40,
    this function will return [(5,20), (30,40)]
    """
    indices = []
    start_index = 1
    end_index = 1
    in_bout = False
    
    for i in range(len(L)):
        if L[i] == 0 and in_bout == False: 
            start_index = i
            in_bout = True
        if (L[i] != 0 or i == len(L)-1) and in_bout == True:
            end_index = i
            in_bout = False
            if i == len(L)-1:
                indices.append((start_index, end_index+1))
            else:
                indices.append((start_index, end_index))
    return indices 
        

### Get day bouts and night bouts

In [17]:
# Need to put this into a function, and make a loop to generate day/night bouts for each fly of 5 in the df
def get_day_night_bouts(df, resp_colnum, sleep_colnum):
    bout_indicies = get_bout_indices(list(df.iloc[:,sleep_colnum]))
    resp_bouts_day = []
    resp_bouts_night = []
    for i in bout_indicies:
        if df.isNight[i[0]:i[1]].mean() >= 0.5:
            resp_bouts_night.append(list(df.iloc[:,resp_colnum])[i[0]:i[1]])
        else:
            resp_bouts_day.append(list(df.iloc[:,resp_colnum])[i[0]:i[1]])
    return resp_bouts_day, resp_bouts_night

### Get sleep profile 


In [24]:
fly1_sleep = df_trim['w1118 BL 6326']
fly1_resp = df_trim.uL_Channel_2
hourly_resp_sleep = []
for i in range(24):
    sleep = fly1_sleep[i*12:i*12+12]
    resp = fly1_resp[i*12:i*12+12]
    num_sleep_blocks = 0
    for i in sleep:
        if i == 0:
            num_sleep_blocks += 1
    sleep_avg = num_sleep_blocks / 12
    hourly_resp_sleep.append((resp.sum(), sleep_avg))
    

In [25]:
hourly_resp_sleep

[(13.666723000000001, 0.0),
 (14.268239, 0.0),
 (14.507468000000003, 0.0),
 (15.105396000000002, 0.0),
 (14.410743, 0.0),
 (15.773512000000004, 0.75),
 (14.073482, 1.0),
 (15.142212999999998, 0.6666666666666666),
 (15.210787, 0.4166666666666667),
 (20.560867199999997, 0.8333333333333334),
 (11.7697673, 0.8333333333333334),
 (13.463753700000002, 0.3333333333333333),
 (11.624898799999997, 0.3333333333333333),
 (10.687663299999999, 1.0),
 (10.567919900000001, 0.75),
 (11.7880435, 0.3333333333333333),
 (16.026454, 0.75),
 (15.986037999999999, 0.75),
 (15.470852, 0.6666666666666666),
 (14.086167, 0.75),
 (13.615887, 0.9166666666666666),
 (13.834914, 0.9166666666666666),
 (14.610688999999999, 0.8333333333333334),
 (14.777592999999998, 0.8333333333333334)]

## Export to .csv

In [51]:
# Now we save this list of lists as a csv
resp_bouts_df = pd.DataFrame(resp_bouts).transpose()
resp_bouts_df.to_csv('test.csv', index=False, header=False)